In [1]:
!unzip testing_dataset.zip

Archive:  testing_dataset.zip
   creating: testing_dataset/
  inflating: __MACOSX/._testing_dataset  
   creating: testing_dataset/cat/
  inflating: __MACOSX/testing_dataset/._cat  
   creating: testing_dataset/donkey/
  inflating: __MACOSX/testing_dataset/._donkey  
   creating: testing_dataset/dog/
  inflating: __MACOSX/testing_dataset/._dog  
   creating: testing_dataset/sheep/
  inflating: __MACOSX/testing_dataset/._sheep  
   creating: testing_dataset/chicken/
  inflating: __MACOSX/testing_dataset/._chicken  
   creating: testing_dataset/monkey/
  inflating: __MACOSX/testing_dataset/._monkey  
   creating: testing_dataset/frog/
  inflating: __MACOSX/testing_dataset/._frog  
   creating: testing_dataset/cow/
  inflating: __MACOSX/testing_dataset/._cow  
   creating: testing_dataset/lion/
  inflating: __MACOSX/testing_dataset/._lion  
   creating: testing_dataset/cat/images/
  inflating: __MACOSX/testing_dataset/cat/._images  
   creating: testing_dataset/cat/sound/
  inflating: __M

In [2]:
!unzip training_dataset.zip

Archive:  training_dataset.zip
   creating: training_dataset/
  inflating: __MACOSX/._training_dataset  
   creating: training_dataset/cat/
  inflating: __MACOSX/training_dataset/._cat  
   creating: training_dataset/donkey/
  inflating: __MACOSX/training_dataset/._donkey  
   creating: training_dataset/dog/
  inflating: __MACOSX/training_dataset/._dog  
   creating: training_dataset/sheep/
  inflating: __MACOSX/training_dataset/._sheep  
   creating: training_dataset/chicken/
  inflating: __MACOSX/training_dataset/._chicken  
   creating: training_dataset/monkey/
  inflating: __MACOSX/training_dataset/._monkey  
   creating: training_dataset/frog/
  inflating: __MACOSX/training_dataset/._frog  
   creating: training_dataset/cow/
  inflating: __MACOSX/training_dataset/._cow  
   creating: training_dataset/lion/
  inflating: __MACOSX/training_dataset/._lion  
   creating: training_dataset/cat/images/
  inflating: __MACOSX/training_dataset/cat/._images  
   creating: training_dataset/cat

In [3]:
!pip install librosa tensorflow numpy

In [4]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import librosa
import numpy as np

## CNN-train image only-test on image

In [5]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the path to the dataset
dataset_path = "training_dataset"

# Parameters
img_size = (128, 128)  # Resize images to a fixed size
batch_size = 16

# Ensure valid image formats are loaded
valid_image_formats = [".jpg", ".jpeg", ".png"]

# Check and count images in all subdirectories
for class_name in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_name, "images")
    if os.path.isdir(class_path):
        print(f"Checking {class_name}...")
        valid_images = [img for img in os.listdir(class_path) if img.lower().endswith(tuple(valid_image_formats))]
        print(f"Found {len(valid_images)} images in {class_name}/images.")

# Image Data Generator for loading and augmenting data
image_gen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    validation_split=0.2  # Split into training and validation
)

# Training data
train_image_data = image_gen.flow_from_directory(
    directory=dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    classes=[cls for cls in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, cls))],
    subset="training",
    class_mode="categorical"
)

# Validation data
val_image_data = image_gen.flow_from_directory(
    directory=dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    classes=[cls for cls in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, cls))],
    subset="validation",
    class_mode="categorical"
)

# Define the CNN model for images
def create_image_cnn(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Dynamically determine the number of classes
num_classes = train_image_data.num_classes

# Create and compile the image classification model
image_model = create_image_cnn((*img_size, 3), num_classes)
image_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
image_model.fit(
    train_image_data,
    validation_data=val_image_data,
    epochs=10
)

# Save the trained model
image_model.save("image_classification_model.h5")


Checking sheep...
Found 15 images in sheep/images.
Checking frog...
Found 15 images in frog/images.
Checking dog...
Found 15 images in dog/images.
Checking cow...
Found 15 images in cow/images.
Checking monkey...
Found 15 images in monkey/images.
Checking lion...
Found 15 images in lion/images.
Checking donkey...
Found 15 images in donkey/images.
Checking chicken...
Found 15 images in chicken/images.
Checking cat...
Found 15 images in cat/images.
Found 108 images belonging to 9 classes.
Found 27 images belonging to 9 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 763ms/step - accuracy: 0.1642 - loss: 2.4068 - val_accuracy: 0.1111 - val_loss: 2.1809
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.2146 - loss: 2.1400 - val_accuracy: 0.2963 - val_loss: 2.1372
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.3397 - loss: 1.9554 - val_accuracy: 0.3704 - val_loss: 2.0666
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.4087 - loss: 1.6485 - val_accuracy: 0.2963 - val_loss: 1.9939
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.6648 - loss: 1.1127 - val_accuracy: 0.3333 - val_loss: 2.2831
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.7505 - loss: 0.7738 - val_accuracy: 0.2222 - val_loss: 3.2426
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.8593 - loss: 0.4883 - val_accuracy: 0.2963 - val_loss: 3.1769
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.9775 - loss: 0.1887 - val_accuracy: 0.3333 - val_loss: 3.0507
Epoch 9/1

In [7]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

# Define the path to the testing dataset
testing_dataset_path = "testing_dataset"

# Parameters
img_size = (128, 128)  # Resize images to a fixed size
batch_size = 16

# Ensure valid image formats are loaded
valid_image_formats = [".jpg", ".jpeg", ".png"]

# Check and count images in all subdirectories of the testing dataset
for class_name in os.listdir(testing_dataset_path):
    class_path = os.path.join(testing_dataset_path, class_name, "images")
    if os.path.isdir(class_path):
        print(f"Checking {class_name}...")
        valid_images = [img for img in os.listdir(class_path) if img.lower().endswith(tuple(valid_image_formats))]
        print(f"Found {len(valid_images)} images in {class_name}/images.")

# Image Data Generator for testing (only rescaling)
test_image_gen = ImageDataGenerator(
    rescale=1./255  # Normalize pixel values
)

# Testing data generator
test_image_data = test_image_gen.flow_from_directory(
    directory=testing_dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    shuffle=False,  # Do not shuffle; keep file order
    class_mode="categorical"
)

# Load the trained model
model_path = "image_classification_model.h5"
print(f"Loading model from {model_path}...")
image_model = load_model(model_path)

# Evaluate the model on the testing data
test_loss, test_accuracy = image_model.evaluate(test_image_data, verbose=1)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

Checking sheep...
Found 25 images in sheep/images.
Checking frog...
Found 20 images in frog/images.
Checking dog...
Found 40 images in dog/images.
Checking cow...
Found 45 images in cow/images.
Checking monkey...
Found 10 images in monkey/images.
Checking lion...
Found 30 images in lion/images.
Checking donkey...
Found 10 images in donkey/images.
Checking chicken...
Found 15 images in chicken/images.
Checking cat...
Found 40 images in cat/images.
Found 235 images belonging to 9 classes.


Loading model from image_classification_model.h5...
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 118ms/step - accuracy: 0.0963 - loss: 6.6585
Test Accuracy: 0.1021
Test Loss: 6.4713


## Use Pretrained Models with Fine-Tuning

In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Parameters
img_size = (128, 128)  # Resize images
batch_size = 16
dataset_path = "training_dataset"

# Data augmentation
image_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Load training and validation data
train_image_data = image_gen.flow_from_directory(
    directory=dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    subset="training",
    class_mode="categorical"
)

val_image_data = image_gen.flow_from_directory(
    directory=dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    subset="validation",
    class_mode="categorical"
)

# Load a pretrained model and fine-tune
base_model = MobileNetV2(input_shape=(*img_size, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the base model

# Add custom layers for your dataset
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(train_image_data.num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_image_data,
    validation_data=val_image_data,
    epochs=10
)

# Save the trained model
model.save("finetuned_mobilenetv2_model.h5")
print("Model saved as 'finetuned_mobilenetv2_model.h5'")


Found 108 images belonging to 9 classes.
Found 27 images belonging to 9 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


7/7 ━━━━━━━━━━━━━━━━━━━━ 23s 2s/step - accuracy: 0.0949 - loss: 2.6149 - val_accuracy: 0.3704 - val_loss: 1.6935
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - accuracy: 0.5641 - loss: 1.4678 - val_accuracy: 0.5185 - val_loss: 1.4250
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - accuracy: 0.7078 - loss: 1.0100 - val_accuracy: 0.6667 - val_loss: 1.0430
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - accuracy: 0.8685 - loss: 0.5682 - val_accuracy: 0.7037 - val_loss: 0.8818
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 0.8254 - loss: 0.4499 - val_accuracy: 0.6296 - val_loss: 1.0186
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - accuracy: 0.9345 - loss: 0.3240 - val_accuracy: 0.6667 - val_loss: 0.8690
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - accuracy: 0.9517 - loss: 0.2552 - val_accuracy: 0.7037 - val_loss: 0.8671
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - accuracy: 0.9123 - loss: 0.3330 - val_accuracy: 0.7037 - val_loss: 0.9102
Epoch 9/10
7

Model saved as 'finetuned_mobilenetv2_model.h5'


In [9]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

# Define the path to the testing dataset
testing_dataset_path = "testing_dataset"

# Parameters
img_size = (128, 128)  # Resize images to a fixed size
batch_size = 16

# Ensure valid image formats are loaded
valid_image_formats = [".jpg", ".jpeg", ".png"]

# Check and count images in all subdirectories of the testing dataset
for class_name in os.listdir(testing_dataset_path):
    class_path = os.path.join(testing_dataset_path, class_name, "images")
    if os.path.isdir(class_path):
        print(f"Checking {class_name}...")
        valid_images = [img for img in os.listdir(class_path) if img.lower().endswith(tuple(valid_image_formats))]
        print(f"Found {len(valid_images)} images in {class_name}/images.")

# Image Data Generator for testing (only rescaling)
test_image_gen = ImageDataGenerator(
    rescale=1./255  # Normalize pixel values
)

# Testing data generator
test_image_data = test_image_gen.flow_from_directory(
    directory=testing_dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    shuffle=False,  # Do not shuffle; keep file order
    class_mode="categorical"
)

# Load the trained model
model_path = "finetuned_mobilenetv2_model.h5"
print(f"Loading model from {model_path}...")
image_model = load_model(model_path)

# Evaluate the model on the testing data
test_loss, test_accuracy = image_model.evaluate(test_image_data, verbose=1)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

Checking sheep...
Found 25 images in sheep/images.
Checking frog...
Found 20 images in frog/images.
Checking dog...
Found 40 images in dog/images.
Checking cow...
Found 45 images in cow/images.
Checking monkey...
Found 10 images in monkey/images.
Checking lion...
Found 30 images in lion/images.
Checking donkey...
Found 10 images in donkey/images.
Checking chicken...
Found 15 images in chicken/images.
Checking cat...
Found 40 images in cat/images.
Found 235 images belonging to 9 classes.
Loading model from finetuned_mobilenetv2_model.h5...


15/15 ━━━━━━━━━━━━━━━━━━━━ 6s 187ms/step - accuracy: 0.8693 - loss: 0.3787
Test Accuracy: 0.8383
Test Loss: 0.4533


## CNN-train audio only-test on audio

In [27]:
import librosa
import numpy as np
import os
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

# Path to audio data
audio_data_path = "training_dataset"

# Parameters
num_classes = len(os.listdir(audio_data_path))  # Number of classes
audio_duration = 15.0  # Duration in seconds
sr = 22050  # Sampling rate

# Preprocessing audio data
def preprocess_audio(audio_dir, classes):
    features, labels = [], []
    for label, cls in enumerate(classes):
        class_path = os.path.join(audio_dir, cls, "sound")
        for file in os.listdir(class_path):
            if file.endswith(".wav"):
                file_path = os.path.join(class_path, file)
                audio, _ = librosa.load(file_path, sr=sr, duration=audio_duration)
                mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)  # Extract MFCC features
                mfccs = np.mean(mfccs.T, axis=0)  # Fixed-size feature vector
                features.append(mfccs)
                labels.append(label)
    return np.array(features), np.array(labels)

# Load audio data
classes = [cls for cls in os.listdir(audio_data_path) if os.path.isdir(os.path.join(audio_data_path, cls))]
audio_features, audio_labels = preprocess_audio(audio_data_path, classes)

# Normalize features and one-hot encode labels
audio_features = np.array(audio_features)
audio_labels = tf.keras.utils.to_categorical(audio_labels, num_classes)

# Split into training and validation
X_train, X_val, y_train, y_val = train_test_split(audio_features, audio_labels, test_size=0.2, random_state=42)

# Define CNN for audio classification
def create_audio_cnn(input_shape, num_classes):
    model = models.Sequential([
        layers.Dense(256, activation='relu', input_shape=input_shape),
        layers.Dropout(0.3),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Create and compile the audio model
audio_model = create_audio_cnn((audio_features.shape[1],), num_classes)
audio_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the audio model
audio_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=16)

# Save the audio model
audio_model.save("audio_classification_model.h5")

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 777ms/step - accuracy: 0.0895 - loss: 30.9318 - val_accuracy: 0.2593 - val_loss: 7.2661
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1170 - loss: 21.1398 - val_accuracy: 0.1481 - val_loss: 6.8633
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1317 - loss: 15.0667 - val_accuracy: 0.2593 - val_loss: 6.3979
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2265 - loss: 13.8885 - val_accuracy: 0.2222 - val_loss: 4.3848
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1499 - loss: 12.7222 - val_accuracy: 0.1852 - val_loss: 4.9124
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2433 - loss: 10.2406 - val_accuracy: 0.2222 - val_loss: 3.6192
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1816 - loss: 7.3550 - val_accuracy: 0.2593 - val_loss: 2.4216
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2640 - loss: 6.2102 - val_accuracy: 0.3704 - val_loss: 2.0788
Epoch 9/10


## Finetuning with a pre-trained model

In [30]:
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow.keras import layers, models
import librosa
import numpy as np
from sklearn.model_selection import train_test_split

# Load YAMNet pretrained model from TensorFlow Hub
yamnet_model_handle = "https://tfhub.dev/google/yamnet/1"
yamnet_model = hub.load(yamnet_model_handle)

# Extract embeddings for audio files using YAMNet
def extract_audio_embeddings(file_path):
    # Load the audio file
    audio, sr = librosa.load(file_path, sr=16000, duration=15.0)  # YAMNet requires 16kHz
    waveform = tf.convert_to_tensor(audio, dtype=tf.float32)
    _, embeddings, _ = yamnet_model(waveform)
    return embeddings.numpy()

# Prepare the dataset
def prepare_audio_data(audio_path, classes):
    features, labels = [], []
    for label, cls in enumerate(classes):
        class_path = os.path.join(audio_path, cls, "sound")
        for file in os.listdir(class_path):
            if file.endswith(".wav"):
                file_path = os.path.join(class_path, file)
                embedding = extract_audio_embeddings(file_path)
                features.append(np.mean(embedding, axis=0))  # Aggregate to fixed-size
                labels.append(label)
    return np.array(features), np.array(labels)

# Load and preprocess audio data
audio_path = "training_dataset"
classes = [cls for cls in os.listdir(audio_path) if os.path.isdir(os.path.join(audio_path, cls))]
audio_features, audio_labels = prepare_audio_data(audio_path, classes)

# One-hot encode labels and split into train and validation sets
audio_labels = tf.keras.utils.to_categorical(audio_labels, len(classes))
X_train, X_val, y_train, y_val = train_test_split(audio_features, audio_labels, test_size=0.2, random_state=42)

# Define a simple classifier for fine-tuning
def create_audio_classifier(input_shape, num_classes):
    model = models.Sequential([
        layers.Dense(256, activation='relu', input_shape=input_shape),
        layers.Dropout(0.3),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Create and compile the model
audio_model = create_audio_classifier((X_train.shape[1],), len(classes))
audio_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = audio_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=16
)

# Save the fine-tuned model
audio_model.save("fine_tuned_audio_model.h5")


Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 388ms/step - accuracy: 0.2171 - loss: 2.2020 - val_accuracy: 0.8148 - val_loss: 1.5392
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5989 - loss: 1.5226 - val_accuracy: 0.9630 - val_loss: 1.0690
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8376 - loss: 1.1071 - val_accuracy: 0.9259 - val_loss: 0.7748
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8739 - loss: 0.8218 - val_accuracy: 0.8889 - val_loss: 0.5877
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8875 - loss: 0.5707 - val_accuracy: 0.8889 - val_loss: 0.4515
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9262 - loss: 0.4363 - val_accuracy: 0.8519 - val_loss: 0.4106
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9697 - loss: 0.2670 - val_accuracy: 0.8889 - val_loss: 0.3738
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9607 - loss: 0.2271 - val_accuracy: 0.8889 - val_loss: 0.3250
Epoch 

#CNN_train audio+image-test on audio+image/image/audio
## CNN Two-Stage Fusion Model

In [35]:
pip install tensorflow librosa numpy scikit-learn

In [36]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import librosa
from sklearn.model_selection import train_test_split

# Parameters
img_size = (128, 128)  # Image size for resizing
audio_duration = 15.0  # Duration in seconds for audio processing
sr = 22050  # Sampling rate for audio files
batch_size = 16
num_classes = 9  # Update based on your dataset structure
dataset_path = "training_dataset"  # Root dataset path

# Helper function: Extract MFCC features from audio
def extract_mfcc(file_path):
    audio, _ = librosa.load(file_path, sr=sr, duration=audio_duration)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)  # 40 MFCC features
    return np.mean(mfccs.T, axis=0)  # Fixed-size representation

# Prepare audio data
def prepare_audio_data(audio_path):
    features, labels = [], []
    classes = [cls for cls in os.listdir(audio_path) if os.path.isdir(os.path.join(audio_path, cls))]
    for label, cls in enumerate(classes):
        class_path = os.path.join(audio_path, cls, "sound")
        for file in os.listdir(class_path):
            if file.endswith(".wav"):
                file_path = os.path.join(class_path, file)
                mfcc = extract_mfcc(file_path)
                features.append(mfcc)
                labels.append(label)
    return np.array(features), np.array(labels), classes

# Prepare image data
def prepare_image_data(image_path):
    image_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
    train_image_data = image_gen.flow_from_directory(
        directory=image_path,
        target_size=img_size,
        batch_size=batch_size,
        subset="training",
        class_mode="categorical"
    )
    val_image_data = image_gen.flow_from_directory(
        directory=image_path,
        target_size=img_size,
        batch_size=batch_size,
        subset="validation",
        class_mode="categorical"
    )
    return train_image_data, val_image_data

# Convert DirectoryIterator to NumPy arrays
def convert_directory_iterator_to_numpy(data_iterator):
    images, labels = [], []
    for i in range(len(data_iterator)):
        batch = data_iterator[i]  # Use indexing to fetch batches
        images.append(batch[0])
        labels.append(batch[1])
    return np.concatenate(images), np.concatenate(labels)

# Load data
audio_path = dataset_path
image_path = dataset_path
audio_features, audio_labels, classes = prepare_audio_data(audio_path)
audio_labels = tf.keras.utils.to_categorical(audio_labels, len(classes))  # One-hot encode labels
X_train_audio, X_val_audio, y_train_audio, y_val_audio = train_test_split(
    audio_features, audio_labels, test_size=0.2, random_state=42
)

train_image_data, val_image_data = prepare_image_data(image_path)
train_images, train_image_labels = convert_directory_iterator_to_numpy(train_image_data)
val_images, val_image_labels = convert_directory_iterator_to_numpy(val_image_data)

# Ensure audio labels match the format of image labels
assert len(train_image_labels) == len(X_train_audio), "Mismatch in train data sizes!"
assert len(val_image_labels) == len(X_val_audio), "Mismatch in validation data sizes!"

# CNN for image data
def create_image_cnn(input_shape):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# CNN for audio data
def create_audio_cnn(input_shape):
    model = models.Sequential([
        layers.Dense(256, activation='relu', input_shape=input_shape),
        layers.Dropout(0.3),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Build the two-stage fusion model
def create_fusion_model(img_shape, audio_shape, num_classes):
    # Image branch
    image_model = create_image_cnn(img_shape)
    image_input = tf.keras.Input(shape=img_shape)
    image_output = image_model(image_input)

    # Audio branch
    audio_model = create_audio_cnn(audio_shape)
    audio_input = tf.keras.Input(shape=audio_shape)
    audio_output = audio_model(audio_input)

    # Fusion
    combined = layers.Concatenate()([image_output, audio_output])
    combined = layers.Dense(128, activation='relu')(combined)
    combined = layers.Dropout(0.3)(combined)
    final_output = layers.Dense(num_classes, activation='softmax')(combined)

    # Combined model
    return tf.keras.Model(inputs=[image_input, audio_input], outputs=final_output)

# Create fusion model
img_shape = (*img_size, 3)  # Image shape (e.g., 128x128x3)
audio_shape = (audio_features.shape[1],)  # Audio shape (e.g., 40 MFCC features)
fusion_model = create_fusion_model(img_shape, audio_shape, len(classes))

# Compile the model
fusion_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the fusion model
history = fusion_model.fit(
    [train_images, X_train_audio],
    train_image_labels,
    validation_data=(
        [val_images, X_val_audio],
        val_image_labels
    ),
    epochs=10,
    batch_size=batch_size
)

# Save the model
fusion_model.save("cnn_two_stage_fusion_model.h5")
print("Model saved as 'cnn_two_stage_fusion_model.h5'")


Found 108 images belonging to 9 classes.
Found 27 images belonging to 9 classes.
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 974ms/step - accuracy: 0.0591 - loss: 2.2230 - val_accuracy: 0.1111 - val_loss: 2.2026
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0778 - loss: 2.2053 - val_accuracy: 0.1111 - val_loss: 2.2015
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0645 - loss: 2.2295 - val_accuracy: 0.0741 - val_loss: 2.2038
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1219 - loss: 2.2034 - val_accuracy: 0.0741 - val_loss: 2.2182
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1652 - loss: 2.2114 - val_accuracy: 0.1111 - val_loss: 2.2028
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1267 - loss: 2.2022 - val_accuracy: 0.1852 - val_loss: 2.2052
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1489 - loss: 2.1904 - val_accuracy: 0.1111 - val_loss: 2.2044
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1794 - loss: 2.1722 - val_accuracy: 0.1852 - val_loss: 2.1765
Epoch 9/10
7

Model saved as 'cnn_two_stage_fusion_model.h5'


In [38]:
import tensorflow as tf
import numpy as np
import librosa
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Parameters
img_size = (128, 128)
audio_duration = 15.0
sr = 22050
batch_size = 16
num_classes = 9
test_dataset_path = "testing_dataset"  # Path to your test dataset
model_path = "cnn_two_stage_fusion_model.h5"

# Helper function: Extract MFCC features from audio
def extract_mfcc(file_path):
    audio, _ = librosa.load(file_path, sr=sr, duration=audio_duration)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
    return np.mean(mfccs.T, axis=0)

# Prepare audio data for testing
def prepare_test_audio_data(audio_path):
    features, labels = [], []
    classes = [cls for cls in os.listdir(audio_path) if os.path.isdir(os.path.join(audio_path, cls))]
    for label, cls in enumerate(classes):
        class_path = os.path.join(audio_path, cls, "sound")
        for file in os.listdir(class_path):
            if file.endswith(".wav"):
                file_path = os.path.join(class_path, file)
                mfcc = extract_mfcc(file_path)
                features.append(mfcc)
                labels.append(label)
    return np.array(features), np.array(labels), classes

# Prepare image data for testing
def prepare_test_image_data(image_path):
    image_gen = ImageDataGenerator(rescale=1./255)
    test_image_data = image_gen.flow_from_directory(
        directory=image_path,
        target_size=img_size,
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=False  # Keep the order for consistency
    )
    return test_image_data

# Convert DirectoryIterator to NumPy arrays
def convert_directory_iterator_to_numpy(data_iterator):
    images, labels = [], []
    for i in range(len(data_iterator)):
        batch = data_iterator[i]
        images.append(batch[0])
        labels.append(batch[1])
    return np.concatenate(images), np.concatenate(labels)

# Load test data
test_audio_features, test_audio_labels, test_classes = prepare_test_audio_data(test_dataset_path)
test_audio_labels = tf.keras.utils.to_categorical(test_audio_labels, len(test_classes))

test_image_data = prepare_test_image_data(test_dataset_path)
test_images, test_image_labels = convert_directory_iterator_to_numpy(test_image_data)

# Load the trained fusion model
print(f"Loading model from {model_path}...")
fusion_model = tf.keras.models.load_model(model_path)

# Ensure test labels match between image and audio
assert len(test_image_labels) == len(test_audio_features), "Mismatch in test data sizes!"

# Evaluate the model on the test set
test_loss, test_accuracy = fusion_model.evaluate(
    [test_images, test_audio_features],
    test_image_labels,
    verbose=1
)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

Found 235 images belonging to 9 classes.


Loading model from cnn_two_stage_fusion_model.h5...
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 88ms/step - accuracy: 0.3728 - loss: 2.0970
Test Accuracy: 0.2170
Test Loss: 2.1367


## Use pretrained- models finetuning

In [39]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub
import librosa
from sklearn.model_selection import train_test_split

# Parameters
img_size = (128, 128)  # Image size for resizing
audio_duration = 15.0  # Duration in seconds for audio processing
sr = 16000  # Sampling rate for audio files
batch_size = 16
num_classes = 9  # Update based on your dataset structure
dataset_path = "training_dataset"  # Root dataset path

# Helper function: Extract audio embeddings using YAMNet
yamnet_model_handle = "https://tfhub.dev/google/yamnet/1"
yamnet_model = hub.load(yamnet_model_handle)

def extract_audio_embeddings(file_path):
    audio, _ = librosa.load(file_path, sr=sr, duration=audio_duration)
    waveform = tf.convert_to_tensor(audio, dtype=tf.float32)
    _, embeddings, _ = yamnet_model(waveform)
    return np.mean(embeddings.numpy(), axis=0)  # Fixed-size embedding

# Prepare audio data
def prepare_audio_data(audio_path):
    features, labels = [], []
    classes = [cls for cls in os.listdir(audio_path) if os.path.isdir(os.path.join(audio_path, cls))]
    for label, cls in enumerate(classes):
        class_path = os.path.join(audio_path, cls, "sound")
        for file in os.listdir(class_path):
            if file.endswith(".wav"):
                file_path = os.path.join(class_path, file)
                embedding = extract_audio_embeddings(file_path)
                features.append(embedding)
                labels.append(label)
    return np.array(features), np.array(labels), classes

# Prepare image data
def prepare_image_data(image_path):
    image_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
    train_image_data = image_gen.flow_from_directory(
        directory=image_path,
        target_size=img_size,
        batch_size=batch_size,
        subset="training",
        class_mode="categorical"
    )
    val_image_data = image_gen.flow_from_directory(
        directory=image_path,
        target_size=img_size,
        batch_size=batch_size,
        subset="validation",
        class_mode="categorical"
    )
    return train_image_data, val_image_data

# Convert DirectoryIterator to NumPy arrays
def convert_directory_iterator_to_numpy(data_iterator):
    images, labels = [], []
    for i in range(len(data_iterator)):
        batch = data_iterator[i]  # Use indexing instead of `next()`
        images.append(batch[0])
        labels.append(batch[1])
    return np.concatenate(images), np.concatenate(labels)

# Load data
audio_path = dataset_path
image_path = dataset_path
audio_features, audio_labels, classes = prepare_audio_data(audio_path)
audio_labels = tf.keras.utils.to_categorical(audio_labels, len(classes))  # One-hot encode labels
X_train_audio, X_val_audio, y_train_audio, y_val_audio = train_test_split(
    audio_features, audio_labels, test_size=0.2, random_state=42
)

train_image_data, val_image_data = prepare_image_data(image_path)
train_images, train_image_labels = convert_directory_iterator_to_numpy(train_image_data)
val_images, val_image_labels = convert_directory_iterator_to_numpy(val_image_data)

# Ensure audio labels match the format of image labels
assert len(train_image_labels) == len(X_train_audio), "Mismatch in train data sizes!"
assert len(val_image_labels) == len(X_val_audio), "Mismatch in validation data sizes!"

# Pretrained MobileNetV2 for Image Data (Fine-Tuning Enabled)
def create_image_branch(input_shape):
    base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights="imagenet")
    base_model.trainable = True  # Enable fine-tuning
    # Fine-tune only the last 4 layers
    for layer in base_model.layers[:-4]:
        layer.trainable = False
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation="relu")
    ])
    return model

# Pretrained YAMNet for Audio Data (Fine-Tuning Enabled)
def create_audio_branch(input_shape):
    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Dense(128, activation="relu"),
        layers.Dropout(0.3),
        layers.Dense(64, activation="relu")
    ])
    return model

# Build the two-stage fusion model with fine-tuning
def create_fusion_model(img_shape, audio_shape, num_classes):
    # Image branch
    image_model = create_image_branch(img_shape)
    image_input = tf.keras.Input(shape=img_shape)
    image_output = image_model(image_input)

    # Audio branch
    audio_model = create_audio_branch(audio_shape)
    audio_input = tf.keras.Input(shape=audio_shape)
    audio_output = audio_model(audio_input)

    # Align dimensions for addition
    image_output_aligned = layers.Dense(64, activation='relu')(image_output)
    audio_output_aligned = layers.Dense(64, activation='relu')(audio_output)

    # Fusion: Use addition
    combined = layers.Add()([image_output_aligned, audio_output_aligned])
    combined = layers.Dense(128, activation='relu')(combined)
    combined = layers.Dropout(0.3)(combined)
    final_output = layers.Dense(num_classes, activation='softmax')(combined)

    return tf.keras.Model(inputs=[image_input, audio_input], outputs=final_output)

# Create fusion model
img_shape = (*img_size, 3)  # Image shape (e.g., 128x128x3)
audio_shape = (audio_features.shape[1],)  # Audio shape (YAMNet embedding size)
fusion_model = create_fusion_model(img_shape, audio_shape, len(classes))

# Compile the model
fusion_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

# Train the fusion model
history = fusion_model.fit(
    [train_images, X_train_audio],
    train_image_labels,
    validation_data=(
        [val_images, X_val_audio],
        val_image_labels
    ),
    epochs=10,
    batch_size=batch_size
)

# Save the model
fusion_model.save("cnn_two_stage_fusion_model_finetuned.h5")
print("Model saved as 'cnn_two_stage_fusion_model_finetuned.h5'")


Found 108 images belonging to 9 classes.
Found 27 images belonging to 9 classes.
Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - accuracy: 0.0477 - loss: 2.3665 - val_accuracy: 0.1852 - val_loss: 2.1330
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2383 - loss: 2.0590 - val_accuracy: 0.1852 - val_loss: 2.0733
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4622 - loss: 1.8357 - val_accuracy: 0.2593 - val_loss: 2.0133
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5537 - loss: 1.6767 - val_accuracy: 0.4074 - val_loss: 1.9403
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6718 - loss: 1.5042 - val_accuracy: 0.4815 - val_loss: 1.8562
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7189 - loss: 1.3741 - val_accuracy: 0.4815 - val_loss: 1.7723
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7585 - loss: 1.2293 - val_accuracy: 0.4815 - val_loss: 1.6801
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/s

Model saved as 'cnn_two_stage_fusion_model_finetuned.h5'


## the above models used concatenation fusion which combines the features from both the image and audio branches by stacking them together along the feature dimension.

The low accuracy of the image+audio fusion CNN compared to the individual CNNs for images and audio alone can be attributed to several potential factors:

1. Over-parameterization of Fusion Network
Fusion networks combine features from two different modalities (image and audio). If the fusion layer adds too many parameters or introduces redundancy in the model, it can lead to overfitting on training data and poor generalization to validation/test data.
Evidence in Your Logs: Training accuracy improves significantly, but validation accuracy remains low. This indicates overfitting.
2. Modality Imbalance
The image features and audio features might not contribute equally to the classification task. If one modality carries more informative features than the other (e.g., images being more relevant than audio or vice versa), the concatenation or fusion might dilute the signal from the dominant modality.
Why This Happens: The network tries to balance between the two modalities, which can reduce performance for one or both modalities.
3. Feature Misalignment
Image and audio features are inherently different. Images are spatially structured (2D arrays), while audio (MFCC) features are 1D or sequential data. Concatenating or combining these features directly might not effectively align their representations for the classification task.
Possible Solution: A more sophisticated fusion mechanism (e.g., attention mechanisms or weighted fusion) can be used to align and emphasize relevant features from both modalities.
4. Data Representation
Audio MFCC Features: While MFCCs are useful, they might not capture enough temporal dynamics if averaged across time. This could lead to under-representation of audio information compared to rich visual features from CNNs.
Image Augmentation: While images are augmented, the same variety might not exist for audio. This imbalance can hinder training the fusion network.